# Transportation

## Problem Statement

### Mathmatical Formulation

$$

\begin{align}
    \text{min} \quad & \sum_{i \in I}\sum_{j \in J} c_{i, j} x_{i, j} \\
    \text{s.t.} \quad & \sum_{j \in J} x_{i, j} \leq b_{i} & \forall \; i \in I \\
    \quad & \sum_{i \in I} x_{i, j} = d_{j} & \forall \; j \in J \\
    & x_{i, j} \geq 0 & \forall \;i \in I, j \in J \\
\end{align}

$$

### Adjusted Constraint to Handle Infesible Demand

$$
\begin{align*}
    \quad & \sum_{i \in I} x_{i, j} \boldsymbol{+ z_{j}} = d_{j} & \forall \; j \in J \tag{3} \\ 
\end{align*}
$$

## Setup

### Libraries

In [1]:
import json
import pathlib

import pandas as pd 
import pyomo.environ as pyo

In [3]:

from plotly import express as px
from plotly import graph_objects as go

## Data

### Read

In [5]:
pathlib.Path.cwd()

WindowsPath('c:/Users/kslad/Documents/projects/optimization/projects/transportation')

In [4]:
with pathlib.Path("input_transp.json").open(mode="r", encoding="utf8") as file:
    input_data = json.load(file)

input_data

{'availabilities': {'S1': 14, 'S2': 26, 'S3': 11},
 'demands': {'C1': 5, 'C2': 13, 'C3': 15, 'C4': 17},
 'costs': [{'from': 'S1', 'to': 'C1', 'value': 10},
  {'from': 'S1', 'to': 'C2', 'value': 5},
  {'from': 'S1', 'to': 'C3', 'value': 20},
  {'from': 'S1', 'to': 'C4', 'value': 12},
  {'from': 'S2', 'to': 'C1', 'value': 12},
  {'from': 'S2', 'to': 'C2', 'value': 7},
  {'from': 'S2', 'to': 'C3', 'value': 12},
  {'from': 'S2', 'to': 'C4', 'value': 19},
  {'from': 'S3', 'to': 'C1', 'value': 6},
  {'from': 'S3', 'to': 'C2', 'value': 12},
  {'from': 'S3', 'to': 'C3', 'value': 16},
  {'from': 'S3', 'to': 'C4', 'value': 17}]}

### Prepare

In [40]:
availabilities = input_data["availabilities"]
demands = input_data["demands"]

# dictionary of costs indexed by tuples (origin, destination)
costs = {
    (c["from"], c["to"]): c["value"]
    for c in input_data["costs"]
}

costs

{('S1', 'C1'): 10,
 ('S1', 'C2'): 5,
 ('S1', 'C3'): 20,
 ('S1', 'C4'): 12,
 ('S2', 'C1'): 12,
 ('S2', 'C2'): 7,
 ('S2', 'C3'): 12,
 ('S2', 'C4'): 19,
 ('S3', 'C1'): 6,
 ('S3', 'C2'): 12,
 ('S3', 'C3'): 16,
 ('S3', 'C4'): 17}

In [41]:
f"supply={sum(availabilities.values())} | demand={sum(demands.values())}"

'supply=51 | demand=50'

In [42]:
# availabilities["S2"] = 23
# f"supply={sum(availabilities.values())} | demand={sum(demands.values())}"

## Model

In [24]:
model = pyo.ConcreteModel()

In [25]:
suppliers = pyo.Set(initialize=availabilities.keys())
customers = pyo.Set(initialize=demands.keys())

model.add_component('suppliers', suppliers)
model.add_component("customers", customers)

model.customers.display()

customers : Size=1, Index=None, Ordered=Insertion
    Key  : Dimen : Domain : Size : Members
    None :     1 :    Any :    4 : {'C1', 'C2', 'C3', 'C4'}


### Parameters

In [26]:
supply = pyo.Param(model.suppliers, initialize=availabilities)
demand = pyo.Param(model.customers, initialize=demands)
costs = pyo.Param(model.suppliers, model.customers, initialize=costs)

model.add_component('supply', supply)
model.add_component("demand", demand)
model.add_component('costs', costs)

model.costs.display()

costs : Size=12, Index=suppliers*customers, Domain=Any, Default=None, Mutable=False
    Key          : Value
    ('S1', 'C1') :    10
    ('S1', 'C2') :     5
    ('S1', 'C3') :    20
    ('S1', 'C4') :    12
    ('S2', 'C1') :    12
    ('S2', 'C2') :     7
    ('S2', 'C3') :    12
    ('S2', 'C4') :    19
    ('S3', 'C1') :     6
    ('S3', 'C2') :    12
    ('S3', 'C3') :    16
    ('S3', 'C4') :    17


### Decision Variables

In [27]:
allocation = pyo.Var(model.suppliers, model.customers, within=pyo.NonNegativeReals)

artificial_demand = pyo.Var(model.customers, within=pyo.NonNegativeReals)
model.add_component('allocation', allocation)
model.add_component('artificial_demand', artificial_demand)

### Constraints

In [28]:
def supplier_availability_constraints(model, supplier):
    return sum(model.allocation[supplier, :]) <= model.supply[supplier]

model.add_component('supplier_availability_constraints', pyo.Constraint(model.suppliers, rule=supplier_availability_constraints))
model.supplier_availability_constraints.pprint()

supplier_availability_constraints : Size=3, Index=suppliers, Active=True
    Key : Lower : Body                                                                          : Upper : Active
     S1 :  -Inf : allocation[S1,C1] + allocation[S1,C2] + allocation[S1,C3] + allocation[S1,C4] :  14.0 :   True
     S2 :  -Inf : allocation[S2,C1] + allocation[S2,C2] + allocation[S2,C3] + allocation[S2,C4] :  26.0 :   True
     S3 :  -Inf : allocation[S3,C1] + allocation[S3,C2] + allocation[S3,C3] + allocation[S3,C4] :  11.0 :   True


In [29]:
def customer_demand_equality_constraints(model, customer):
    return sum(model.allocation[:, customer]) + model.artificial_demand[customer] == model.demand[customer]


model.add_component('customer_demand_equality_constraints', pyo.Constraint(model.customers, rule=customer_demand_equality_constraints))
model.customer_demand_equality_constraints.pprint()

customer_demand_equality_constraints : Size=4, Index=customers, Active=True
    Key : Lower : Body                                                                              : Upper : Active
     C1 :   5.0 : allocation[S1,C1] + allocation[S2,C1] + allocation[S3,C1] + artificial_demand[C1] :   5.0 :   True
     C2 :  13.0 : allocation[S1,C2] + allocation[S2,C2] + allocation[S3,C2] + artificial_demand[C2] :  13.0 :   True
     C3 :  15.0 : allocation[S1,C3] + allocation[S2,C3] + allocation[S3,C3] + artificial_demand[C3] :  15.0 :   True
     C4 :  17.0 : allocation[S1,C4] + allocation[S2,C4] + allocation[S3,C4] + artificial_demand[C4] :  17.0 :   True


### Objective

In [30]:
def artificial_objective_function(model):
    return sum(model.artificial_demand[:])

def objective_function(model):
    total_cost = sum(
        model.costs[supplier,customer] * model.allocation[supplier,customer]
        for supplier in model.suppliers
        for customer in model.customers
    )
    return total_cost

model.add_component('artificial_objective_function', pyo.Objective(rule=artificial_objective_function, sense=pyo.minimize))
model.add_component('objective_function', pyo.Objective(rule=objective_function, sense=pyo.minimize))

## Solve

In [31]:
model.objective_function.deactivate()
model.artificial_objective_function.activate()

In [43]:
solver = pyo.SolverFactory("appsi_highs")
result = solver.solve(model)
print(result)


Problem: 
- Lower bound: 526.0
  Upper bound: 526.0
  Number of objectives: 1
  Number of constraints: 0
  Number of variables: 0
  Sense: minimize
Solver: 
- Status: ok
  Termination condition: optimal
  Termination message: TerminationCondition.optimal
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



In [33]:
artificial_demand_adjustment = model.artificial_objective_function()
model.objective_function()

672.0

In [34]:
def artificial_demand_adjustment_constraints(model):
    return sum(model.artificial_demand[:]) <= artificial_demand_adjustment

model.add_component('artificial_demand_adjustment_constraints', pyo.Constraint(rule=artificial_demand_adjustment_constraints))
model.artificial_demand_adjustment_constraints.pprint()

artificial_demand_adjustment_constraints : Size=1, Index=None, Active=True
    Key  : Lower : Body                                                                                          : Upper : Active
    None :  -Inf : artificial_demand[C1] + artificial_demand[C2] + artificial_demand[C3] + artificial_demand[C4] :   0.0 :   True


In [35]:
model.artificial_objective_function.deactivate()
model.objective_function.activate()

In [36]:
solver = pyo.SolverFactory("appsi_highs")
result = solver.solve(model)
print(result)

[{'Lower bound': 526.0, 'Upper bound': 526.0, 'Number of objectives': 1, 'Number of constraints': 0, 'Number of variables': 0, 'Sense': 'minimize'}]

### Solution

In [67]:
solution = [
    {"supplier": supplier, "customer": customer, "amount": amount}
    for (supplier, customer), amount in model.allocation.extract_values().items()
]

df = pd.DataFrame(solution).pivot(
    index="supplier", columns="customer", values="amount"
)
df

customer,C1,C2,C3,C4
supplier,,,,
S1,0.00,2.00,0.00,12.00
S2,0.00,11.00,15.00,0.00
S3,5.00,0.00,0.00,5.00


In [74]:
df.plot.bar()